In [1]:
import sqlite3
import pandas as pd
import hashlib

In [4]:
orders_df = pd.read_csv("orders.csv")
items_df = pd.read_csv("items.csv")

In [9]:
conn = sqlite3.connect("orders.db")

orders_df.to_sql("orders", conn, if_exists="replace", index=False)
items_df.to_sql("items", conn, if_exists="replace", index=False)


7

In [10]:
cur = conn.cursor()
cur.execute("""
CREATE TABLE IF NOT EXISTS orders_history (
    order_id INTEGER,
    user_id INTEGER,
    item_id INTEGER,
    total_price REAL,
    hash_diff TEXT,
    change_date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (order_id, change_date)
);
""")
conn.commit()


In [13]:
def calculate_hash(order):
    """Создаёт хеш-код от user_id, item_id и стоимости заказа"""
    order_string = f"{order['user_id']}{order['item_id']}{order['total_price']}"
    return hashlib.md5(order_string.encode()).hexdigest()


In [19]:
def get_actual_price(item_id, order_date):
    """Возвращает актуальную цену товара на момент заказа"""
    query = """
    SELECT price FROM items 
    WHERE item_id = ? AND update_date <= ? 
    ORDER BY update_date DESC 
    LIMIT 1
    """
    cur.execute(query, (item_id, order_date))
    result = cur.fetchone()
    return result[0] if result else None  


In [20]:
def process_order(order):
    """Добавляет заказ в базу, если он изменился"""
    
    order_id = order["order_id"]
    
    actual_price = get_actual_price(order["item_id"], order["order_date"])
    if actual_price is None:
        print(f"Ошибка: не найдена цена для item_id {order['item_id']} на {order['order_date']}")
        return
    
    order["total_price"] = actual_price  
    new_hash = calculate_hash(order)  

    cur.execute("""
        SELECT hash_diff FROM orders_history 
        WHERE order_id = ? 
        ORDER BY change_date DESC 
        LIMIT 1
    """, (order_id,))
    
    last_hash = cur.fetchone()

    if last_hash and last_hash[0] == new_hash:
        print(f"Заказ {order_id} не изменился, пропускаем.")
        return

    cur.execute("""
        INSERT INTO orders_history (order_id, user_id, item_id, total_price, hash_diff)
        VALUES (?, ?, ?, ?, ?)
    """, (order_id, order["user_id"], order["item_id"], order["total_price"], new_hash))
    
    conn.commit()
    print(f"Заказ {order_id} обновлён!")


In [23]:
orders_stream = pd.read_sql_query("SELECT * FROM orders", conn)

orders_list = orders_stream.to_dict(orient="records")

for order in orders_list:
    process_order(order)


Заказ 1 не изменился, пропускаем.
Заказ 2 не изменился, пропускаем.
Заказ 3 не изменился, пропускаем.
Заказ 4 не изменился, пропускаем.
Заказ 5 не изменился, пропускаем.
Заказ 6 не изменился, пропускаем.


In [25]:
df = pd.read_sql_query("SELECT * FROM orders_history", conn)

df

,order_id,user_id,item_id,total_price,hash_diff,change_date
0,1,1,1,10.0,2cbca44843a864533ec05b321ae1f9d1,2025-02-26 08:19:07
1,2,2,2,2.0,bcbe3365e6ac95ea2c0343a2395834dd,2025-02-26 08:19:07
2,3,1,3,5.0,7f1de29e6da19d22b51c68001e7e0e54,2025-02-26 08:19:07
3,4,3,2,7.0,b83aac23b9528732c23cc7352950e880,2025-02-26 08:19:07
4,5,2,1,10.0,c3535febaff29fcb7c0d20cbe94391c7,2025-02-26 08:19:07
5,6,1,1,5.0,2b44928ae11fb9384c4cf38708677c48,2025-02-26 08:19:08
